In [1]:
import numpy as np

def read_energy_distance_data(file_path):
    data = []

    with open(file_path, 'r') as file:
        for line in file:
            energy, x, y, z = map(float, line.strip().split())
            data.append((energy, np.array([x, y, z])))

    return data

def project_energy_along_axis(energy_framework, axis):
    axis = np.array(axis) / np.linalg.norm(axis)
    projected_energy = sum(energy * np.dot(direction, axis) for energy, direction in energy_framework)
    return projected_energy

# Read energy and distance data
file_path = 'your_energy_distance_data_file.txt'  # Replace this with your file path
energy_framework = read_energy_distance_data(file_path)

# Calculate energy value along a specific axis
axis = [1, 0, 0]  # Replace this with your desired axis
projected_energy = project_energy_along_axis(energy_framework, axis)
print("Projected energy along the axis:", projected_energy)

FileNotFoundError: [Errno 2] No such file or directory: 'your_energy_distance_data_file.txt'

In [45]:
import numpy as np
from ase import Atoms
from ase.io import read
from ase.build import make_supercell
from ase.geometry import find_mic

def read_cif_file(file_path):
    return read(file_path)

def center_of_gravity(molecule):
    weighted_positions = molecule.get_masses()[:, None] * molecule.get_positions()
    total_mass = molecule.get_masses().sum()
    return weighted_positions.sum(axis=0) / total_mass

def get_surrounding_molecules(crystal_structure, molecule_indices, central_molecule_indices, supercell_matrix):
    supercell = make_supercell(crystal_structure, supercell_matrix)
    
    central_molecule = Atoms([supercell[i] for i in central_molecule_indices])
    surrounding_molecules = [Atoms([supercell[j] for j, atom in enumerate(supercell) if j not in central_molecule_indices])]
    
    return central_molecule, surrounding_molecules

def calculate_vectors_between_centers_of_gravity(central_molecule, surrounding_molecules):
    vectors = []
    central_molecule_cog = center_of_gravity(central_molecule)

    for surrounding_molecule in surrounding_molecules:
        surrounding_molecule_cog = center_of_gravity(surrounding_molecule)
        vector = find_mic(surrounding_molecule_cog - central_molecule_cog, central_molecule.cell)[0]
        vectors.append(vector)

    return vectors

# Read CIF file
file_path = './SA/Rac-2-naph_20degree.major.cif'  # Replace this with your CIF file path
crystal_structure = read_cif_file(file_path)

mol1 = []
mol2 = []
mol3 = []
mol4 = []
for i in range(len(crystal_structure.get_atomic_numbers())):
    if i%4==0:
        mol1.append(i)
    elif i%4==1:
        mol2.append(i)
    elif i%4==2:
        mol3.append(i)
    elif i%4==3:
        mol4.append(i)

# Define molecule indices in the unit cell
molecule_indices = [
    mol1,
    mol2,
    mol3,
    mol4,
]

# Select a molecule and generate surrounding molecules
central_molecule_indices = mol1  # Replace this with the indices of the central molecule you want to select
supercell_matrix = np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]])  # Modify this to control the size of the supercell
central_molecule, surrounding_molecules = get_surrounding_molecules(crystal_structure, molecule_indices, central_molecule_indices, supercell_matrix)

# Calculate the center of gravity of each molecule
central_molecule_cog = center_of_gravity(central_molecule)
surrounding_molecules_cog = [center_of_gravity(molecule) for molecule in surrounding_molecules]

# Calculate vectors between the center of gravity of the central molecule and surrounding molecules
vectors = calculate_vectors_between_centers_of_gravity(central_molecule, surrounding_molecules)
print("Vectors between centers of gravity:")
for i, vector in enumerate(vectors):
    print(f"Vector {i + 1}: {vector}")

Vectors between centers of gravity:
Vector 1: [ 2.9710026   3.26490161 13.8457846 ]


C:\Users\s0928\anaconda3\envs\wallach2\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'monoclinic' is not interpreted for space group Spacegroup(14, setting=1). This may result in wrong setting!
  setting_name, spacegroup))


In [25]:
mol1 = []
for i in range(len(crystal_structure.get_atomic_numbers())):
    if i%4==0:
        mol1.append(i)

In [35]:
supercell = make_supercell(crystal_structure, supercell_matrix)

In [37]:
central_molecule = Atoms([supercell[i] for i in central_molecule_indices])

In [46]:
vectors

[array([ 2.9710026 ,  3.26490161, 13.8457846 ])]

In [50]:
from ase.data import covalent_radii
from scipy.spatial.distance import pdist, squareform
import networkx as nx

def covalent_radii_sum(atom1, atom2):
    return covalent_radii[atom1.number] + covalent_radii[atom2.number]

In [51]:
def create_connectivity_graph(atoms, tolerance=1.2):
    connectivity_graph = nx.Graph()
    n_atoms = len(atoms)
    connectivity_graph.add_nodes_from(range(n_atoms))

    distance_matrix = squareform(pdist(atoms.positions))

    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            distance = distance_matrix[i, j]
            bond_cutoff = covalent_radii_sum(atoms[i], atoms[j]) * tolerance
            if distance <= bond_cutoff:
                connectivity_graph.add_edge(i, j)

    return connectivity_graph

In [52]:
filename = './SA/Rac-2-naph_20degree.major.cif'
structure = read(filename)
connectivity_graph = create_connectivity_graph(structure)

In [53]:
molecules = list(nx.connected_components(connectivity_graph))

# Get vectors and calculate angles between specific directions
**Instruction**  
The aim of this notebook is to get vectors from the center molecule to surrounding molecules and to calculate angles between specific directions.  
The execution of the following code requires atomic coordinates of molecular clucster and cell vector information (they are stored `.csv` and `.cif` in the below case).  

**Contents**  
1. Preparation of calculation
1. Calculate the vector of selected center molecule
1. Calculate vectors, distancec, and angles

## 1. Preparation of calculation

In [77]:
import math

def dot_product(a, b):
    return sum([a[i] * b[i] for i in range(len(a))])

def norm(a):
    return math.sqrt(sum([a[i] * a[i] for i in range(len(a))]))

def angle_between_vectors(a, b):
    cosine_angle = dot_product(a, b) / (norm(a) * norm(b))
    cosine_angle = round(cosine_angle, ndigits=5)
    angle = math.acos(cosine_angle) * 180 / math.pi
    return angle

def euclidean_distance(a, b):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))]))

def perpendicular_vector(v1, v2):
    return np.cross(v1, v2)

In [208]:
import pandas as pd
import numpy as np
from ase import io
# name = 'S-1_-80'
df = pd.read_csv(f'./SA/coordinate_{name}_b.csv') # Change path for atomic coordinates of molecular cluster
cryst = io.read(f'./SA/Structure_{name}_major.cif') # Change path for cif file

C:\Users\s0928\anaconda3\envs\wallach2\lib\site-packages\ase\io\cif.py:404: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  setting_name, spacegroup))


In [209]:
df_rev = df.iloc[1071:, :] # 1071, 946 # Please input the starting index of the molecular clucter
df_rev.head(2)

,--------------------------------------------------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
1071,H25B,H,-0.4438,1.7322,2.2013,1
1072,C25A,C,-0.9446,1.8862,1.2534,1


## 2. Calculate the vector of selected center molecule

In [210]:
n_atoms_per_mol = 62
n_mols = 17
centers_list = []

for i in range(n_mols):
    if i==0:
        coordinates = df_rev.iloc[:n_atoms_per_mol, 1:-1]
    else:
        coordinates = df_rev.iloc[n_atoms_per_mol*i+2*i:n_atoms_per_mol*(i+1)+2*i, 1:-1]
    excluded_category = 'P'
    coordinates = coordinates[coordinates['Unnamed: 1'] != excluded_category]
    center = [round(coordinates.iloc[:,1].astype(float).mean(),4),
              round(coordinates.iloc[:,2].astype(float).mean(),4),
              round(coordinates.iloc[:,3].astype(float).mean(),4)]
    centers_list.append(center)

sel_mol = df.iloc[4:4+n_atoms_per_mol, 1:-1]
sel_mol = sel_mol[sel_mol['Unnamed: 1'] != excluded_category]
sel_mol_center = [round(sel_mol.iloc[:,1].astype(float).mean(),4),
                  round(sel_mol.iloc[:,2].astype(float).mean(),4),
                  round(sel_mol.iloc[:,3].astype(float).mean(),4)]
sel_mol_center

[5.6023, 6.8549, 2.4835]

## 3. Calculate vectors, distancec, and angles

In [211]:
distances = [round(euclidean_distance(sel_mol_center, sublist), 2) for sublist in centers_list]
vec_mols_diff = [list(np.array(sel_mol_center)-np.array(sublist)) for sublist in centers_list]

vec_along_a = cryst.cell[0]
vec_along_b = cryst.cell[1]
vec_along_c = cryst.cell[2]
vec_perp_100 = perpendicular_vector(vec_along_b, vec_along_c)
vec_perp_010 = perpendicular_vector(vec_along_a, vec_along_c)
vec_perp_001 = perpendicular_vector(vec_along_a, vec_along_b)

angles_along_a = [round(angle_between_vectors(vec_along_a, sublist),2) for sublist in vec_mols_diff]
angles_along_b = [round(angle_between_vectors(vec_along_b, sublist),2) for sublist in vec_mols_diff]
angles_along_c = [round(angle_between_vectors(vec_along_c, sublist),2) for sublist in vec_mols_diff]
angles_perp_100 = [round(angle_between_vectors(vec_perp_100, sublist),2) for sublist in vec_mols_diff]
angles_perp_010 = [round(angle_between_vectors(vec_perp_010, sublist),2) for sublist in vec_mols_diff]
angles_perp_001 = [round(angle_between_vectors(vec_perp_001, sublist),2) for sublist in vec_mols_diff]

C:\Users\s0928\anaconda3\envs\wallach2\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [212]:
summary = pd.DataFrame({
    'vec_x': [sublist[0] for sublist in vec_mols_diff],
    'vec_y': [sublist[1] for sublist in vec_mols_diff],
    'vec_z': [sublist[2] for sublist in vec_mols_diff],
    'distance_from_center': distances,
    'angle_along_a': angles_along_a,
    'angle_along_b': angles_along_b,
    'angle_along_c': angles_along_c,
    'angle_perp_100': angles_perp_100,
    'angle_perp_010': angles_perp_010,
    'angle_perp_001': angles_perp_001,
})
summary

,vec_x,vec_y,vec_z,distance_from_center,angle_along_a,angle_along_b,angle_along_c,angle_perp_100,angle_perp_010,angle_perp_001
0,6.4935,9.9160,0.0000,11.85,56.78,31.27,83.10,58.80,146.34,90.00
1,6.1564,0.0000,0.0000,6.16,0.00,88.05,86.24,4.08,90.00,90.00
2,5.0814,4.7526,-8.4603,10.95,62.36,63.30,133.84,60.25,120.64,140.57
3,4.7443,-5.1634,-8.4603,10.99,64.42,117.06,141.57,60.35,67.06,140.35
4,0.0000,0.0000,0.0000,0.00,NaN,NaN,NaN,NaN,NaN,NaN
5,-1.0750,4.7526,-8.4603,9.76,96.32,61.13,144.94,94.14,124.87,150.06
6,-0.1288,-3.1934,10.9720,11.43,90.65,106.24,22.44,93.54,67.99,16.24
7,0.3371,9.9160,0.0000,9.92,88.05,0.00,84.10,90.00,173.90,90.00
8,-1.4121,-5.1634,-8.4603,10.01,98.11,121.35,154.17,94.04,64.67,147.68
9,0.2083,6.7226,10.9720,12.87,89.07,58.49,25.84,93.14,115.70,31.51
